In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("authors.csv")

In [2]:
df.columns

Index(['Timestamp', 'Email Address', 'First Name', 'Middle Initial(s)',
       'Last Name', 'Highest degree (PhD, MD, etc)', 'Working Group(s)',
       'Department, Institution (e.g. Psychiatric Genetics, QIMR Berghofer Medical Research Institute)',
       'Street address (e.g. 300 Herston Rd)', 'City (e.g. Brisbane)',
       'ZIP or Postal Code', 'State or Province  (if applicable)', 'Country',
       'Department, Institution', 'Street address', 'City',
       'ZIP or Postal Code.1', 'State or Province  (if applicable).1',
       'Country.1', 'Department, Institution ', 'Street address ', 'City ',
       'ZIP or Postal Code.2', 'State or Province  (if applicable).2',
       'Country.2',
       '4+ Affiliations (please leave an empty line between affiliations) ',
       'Funding Acknowledgements', 'Conflict of Interest/Disclosures'],
      dtype='object')

In [3]:
df_deparments_and_address = df.iloc[:, [7, 8, 9, 10, 11, 12]]

# Cleaning for Record cleaning

In [4]:
from recordlinkage.preprocessing import clean

In [5]:
df_cleaned = df_deparments_and_address.apply(clean, axis=1)

Rename the name of columns

In [6]:
df_cleaned.columns = [
    'department', 'street', 'city', 'zip', 'state', 'country'
]

# Index and check

In [7]:
import recordlinkage

In [8]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(df_cleaned)

/Users/xingyuwei/anaconda3/lib/python3.7/site-packages/recordlinkage/index.py:60: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  levels=levels, labels=labels, verify_integrity=False)


In [9]:
print(len(df_cleaned), len(candidate_links))

174 15051


# Compare records

In [14]:
compare_cl = recordlinkage.Compare()

In [15]:
for column_name in df_cleaned.columns:
    compare_cl.string(column_name, column_name, 'jarowinkler')

In [16]:
df_cleaned.iloc[0:3, :]

,department,street,city,zip,state,country
0,imaging genetics center of usc,4676 admiralty way,marina del rey,90292,NaN,usa
1,department of neurology university of utah,175 n medical dr,salt lake city,84132,ut,united states
2,imaging genetics center mark and mary stevens ...,4676 admiralty way,marina del rey,90292,california,usa


In [17]:
pair_scoring = compare_cl.compute(candidate_links, df_cleaned, df_cleaned)

In [30]:
kmeans = recordlinkage.KMeansClassifier()
result_k_means  = kmeans.fit_predict(pair_scoring)

In [21]:
result_k_means


### using ECM Classifier

In [22]:
ecm = recordlinkage.ECMClassifier(binarize=0.8)
result_ecm = ecm.fit(pair_scoring)

### using Sklearn classifier

In [23]:
import sklearn

In [28]:
k_means_classifier = sklearn.cluster.KMeans(n_clusters=2)


In [29]:
result_k_means = k_means_classifier.fit_predict(pair_scoring)
